In [2]:
!pip install pandas 


In [4]:
!python -m pip install --upgrade pip



     ---------------------------------------- 1.8/1.8 MB 1.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2


In [5]:
!pip install pyarrow

   ---------------------------------------- 25.3/25.3 MB 1.5 MB/s eta 0:00:00


In [6]:
import pandas as pd

# Read Parquet file
df_yellow_jan23 = pd.read_parquet(r'C:\Users\HP\Downloads\yellow_tripdata_2023-01.parquet', engine='pyarrow')  
# Read Parquet file
df_yellow_dec23 = pd.read_parquet(r'C:\Users\HP\Downloads\yellow_tripdata_2023-12.parquet', engine='pyarrow') 

In [7]:
# Rename the column in one dataset (example: renaming in the December dataset)
df_yellow_dec23 = df_yellow_dec23.rename(columns={"Airport_fee": "airport_fee"})

In [8]:
# Combine datasets
combined_df_yellow = pd.concat([df_yellow_jan23, df_yellow_dec23], ignore_index=True)

In [9]:
combined_df_yellow2 = combined_df_yellow.drop_duplicates().reset_index(drop=True)

In [10]:
import pandas as pd

def create_datetime_dim(data, pickup_col, dropoff_col):
    """
    Creates a datetime dimension DataFrame from the given dataset.

    Parameters:
    - data (pd.DataFrame): The input dataset containing datetime columns.
    - pickup_col (str): The name of the pickup datetime column.
    - dropoff_col (str): The name of the dropoff datetime column.

    Returns:
    - pd.DataFrame: A DataFrame with datetime components extracted for both pickup and dropoff datetimes.
    """
    datetime_dim = data[[pickup_col, dropoff_col]].drop_duplicates().reset_index(drop=True)
    
    # Extract datetime components for the pickup datetime
    datetime_dim['pick_hour'] = datetime_dim[pickup_col].dt.hour
    datetime_dim['pick_day'] = datetime_dim[pickup_col].dt.day
    datetime_dim['pick_month'] = datetime_dim[pickup_col].dt.month
    datetime_dim['pick_year'] = datetime_dim[pickup_col].dt.year
    datetime_dim['pick_weekday'] = datetime_dim[pickup_col].dt.weekday
    datetime_dim['pick_quarter'] = datetime_dim[pickup_col].dt.quarter

    # Extract datetime components for the dropoff datetime
    datetime_dim['drop_hour'] = datetime_dim[dropoff_col].dt.hour
    datetime_dim['drop_day'] = datetime_dim[dropoff_col].dt.day
    datetime_dim['drop_month'] = datetime_dim[dropoff_col].dt.month
    datetime_dim['drop_year'] = datetime_dim[dropoff_col].dt.year
    datetime_dim['drop_weekday'] = datetime_dim[dropoff_col].dt.weekday
    datetime_dim['drop_quarter'] = datetime_dim[dropoff_col].dt.quarter

    return datetime_dim


In [11]:
datetime_dim_y = create_datetime_dim(combined_df_yellow2, 'tpep_pickup_datetime', 'tpep_dropoff_datetime')
datetime_dim_y['datetime_id'] = datetime_dim_y.index
datetime_dim_y = datetime_dim_y[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday','pick_quarter',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday','drop_quarter']]
datetime_dim_y.head()

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,pick_quarter,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday,drop_quarter
0,0,2023-01-01 00:32:10,0,1,1,2023,6,1,2023-01-01 00:40:36,0,1,1,2023,6,1
1,1,2023-01-01 00:55:08,0,1,1,2023,6,1,2023-01-01 01:01:27,1,1,1,2023,6,1
2,2,2023-01-01 00:25:04,0,1,1,2023,6,1,2023-01-01 00:37:49,0,1,1,2023,6,1
3,3,2023-01-01 00:03:48,0,1,1,2023,6,1,2023-01-01 00:13:25,0,1,1,2023,6,1
4,4,2023-01-01 00:10:29,0,1,1,2023,6,1,2023-01-01 00:21:19,0,1,1,2023,6,1


In [12]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = combined_df_yellow2[['RatecodeID']].drop_duplicates().reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]
rate_code_dim.head()

,rate_code_id,RatecodeID,rate_code_name
0,0,1.0,Standard rate
1,1,2.0,JFK
2,2,4.0,Nassau or Westchester
3,3,99.0,NaN
4,4,5.0,Negotiated fare


In [13]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = combined_df_yellow2[['payment_type']].drop_duplicates().reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]
payment_type_dim.head()

,payment_type_id,payment_type,payment_type_name
0,0,2,Cash
1,1,1,Credit card
2,2,4,Dispute
3,3,3,No charge
4,4,0,NaN


In [15]:
# Optimize data types
def optimize_dtypes(df):
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    return df

# Apply to all datasets
combined_df_yellow2 = optimize_dtypes(combined_df_yellow2)
rate_code_dim = optimize_dtypes(rate_code_dim)
datetime_dim_y = optimize_dtypes(datetime_dim_y)
payment_type_dim = optimize_dtypes(payment_type_dim)

In [16]:
import pandas as pd

# Assuming rate_code_dim, datetime_dim_y, payment_type_dim are your DataFrame objects
# Save to CSV files in your local directory

rate_code_dim.to_csv(r'C:\Users\HP\Downloads\Documents\rate_code_dim.csv', index=False)
datetime_dim_y.to_csv(r'C:\Users\HP\Downloads\Documents\datetime_dim_y.csv', index=False)
payment_type_dim.to_csv(r'C:\Users\HP\Downloads\Documents\payment_type_dim.csv', index=False)

print("Data exported successfully!")


Data exported successfully!


In [17]:
import pandas as pd



# Save the DataFrame as a Parquet file
parquet_file = "datetime_dim_y.parquet"
datetime_dim_y.to_parquet(parquet_file, engine='pyarrow', index=False)  # Use 'fastparquet' instead of 'pyarrow' if preferred

print(f"Converted datetime_dim_y to {parquet_file}")

Converted datetime_dim_y to datetime_dim_y.parquet


In [18]:
import shutil

# Path to the Parquet file in your Jupyter Notebook environment
source_path = "datetime_dim_y.parquet"

# Destination path on your local computer
destination_path = r"C:\Users\HP\Downloads\Documents\datetime_dim_y.parquet"

# Copy the file to the desired location
shutil.copy(source_path, destination_path)

print(f"File saved to {destination_path}")


File saved to C:\Users\HP\Downloads\Documents\datetime_dim_y.parquet
